In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from utils import *
from model import *

2023-03-21 19:28:55.998801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 19:28:56.449746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 19:28:56.449810: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 19:28:57.583736: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# load all test data
x_test,y_test = load_pkl_data(test_pkl)
print("%d Validation Data loaded"%(len(x_test)))

3469 Validation Data loaded


In [4]:
# load label maps
label2ind = load_dict_from_json(label2ind_json)
ind2label = load_dict_from_json(ind2label_json)
print("Labels loaded")
nc = len(label2ind.keys())
print("Total %d classes"%nc)

Labels loaded
Total 3 classes


In [5]:
model = AttentionClassifier(nc)
print("Model created")

2023-03-21 19:29:00.051651: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-21 19:29:00.052102: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 19:29:00.052450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-21 19:29:00.052725: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-21 19:29:00.052994: W tensorflow/c

Model created


In [6]:
model.load_model("model/")

Model restored weights from model/ckpt-10


In [44]:
# take a random data from test
n_test = len(x_test)
rand_idx = np.random.randint(0,n_test)
print("Random Index %d"%rand_idx)
test_tweet = x_test[rand_idx]
test_label = y_test[rand_idx]
print(test_tweet)
print("Label: %s"%test_label)

Random Index 3169
danieebro all these bitches fake if you ask me
Label: offensive_language


In [45]:
word_attention = model.predict_with_explain(test_tweet,ind2label)
for w in word_attention:
    print(w[0],w[1])

[CLS] 0.26615798473358154
danieebro 0.14215525425970554
all 0.021965084597468376
these 0.004194268025457859
bitches 0.04056389629840851
fake 0.03019515983760357
if 0.07169847190380096
you 0.008941817097365856
ask 0.12221364676952362
me 0.003794984659180045
[SEP] 0.28811943531036377


In [15]:
decoded = model.encoder_layer.preprocess_layer.decode(tf.squeeze(words,-1)) 
print(decoded)

[CLS] angelis yep which is messier more dangerous more environmentally unfriendly than their retarded new spouts [SEP]


In [34]:
sub_word_ids = model.encoder_layer.preprocess_layer.encode(test_tweet)
sub_word_dict = {}
for i,sw in enumerate(sub_word_ids):
    sub_word_dict[i] = sw
# print(sub_word_dict)
word_spans = []
subword_list = []
for k in sub_word_dict.keys():
    sid = sub_word_dict[k]
    sub_word = model.encoder_layer.preprocess_layer.convert_ids_to_tokens(sid)
    if "##" in sub_word:
        word_spans[-1].append(k)
        subword_list[-1].append(sub_word)
    else:
        word_spans.append([k])
        subword_list.append([sub_word])
    # print(k,sid,sub_word)
for i,s in zip(word_spans,subword_list):
    print(i,s)
# all_spans = []
# for id_ in word_ids:
#     sub_word = model.encoder_layer.preprocess_layer.convert_ids_to_tokens(id_)
#     if "##" in sub_word:
#         all_spans[-1].append(id_)
#     else:
#         all_spans.append([id_])
#     # print(id_,sub_word)
# word_span = {}
# for i,sp in enumerate(all_spans):
#     sub_list = []
#     sub_idx = []
#     for s in sp:
#         sub_word = model.encoder_layer.preprocess_layer.convert_ids_to_tokens(s)
#         sub_list.append(sub_word.replace("##",""))
#         sub_idx.append(s)
#     wd = ''.join(sub_list)
#     word_span[str(i)] = {"word":wd,"subword_index":sub_idx}
# print(word_span)


[0] ['[CLS]']
[1, 2] ['angel', '##is']
[3] ['yep']
[4] ['which']
[5] ['is']
[6, 7] ['mess', '##ier']
[8] ['more']
[9] ['dangerous']
[10] ['more']
[11] ['environmentally']
[12, 13, 14, 15] ['un', '##fr', '##ien', '##dly']
[16] ['than']
[17] ['their']
[18, 19, 20] ['re', '##tar', '##ded']
[21] ['new']
[22, 23] ['sp', '##outs']
[24] ['[SEP]']
